### 1. Task 1

Read data:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def read_conllu(path):
    sentences = []
    with open(path, encoding='utf-8') as f:
        tokens = []
        for line in f:
            line = line.rstrip('\n')
            if not line:
                if tokens:
                    sentences.append(tokens)
                tokens = []
                continue
            if line.startswith('#'):
                continue
            cols = line.split('\t')
            if len(cols) != 10:
                raise ValueError("Expected 10 columns, got %d: %r" % (len(cols), line))
            token = (cols[1], cols[3])
            tokens.append(token)
    return sentences

In [3]:
train_path = '/content/drive/MyDrive/UD_English-EWT/en_ewt-ud-train.conllu'
dev_path = '/content/drive/MyDrive/UD_English-EWT/en_ewt-ud-dev.conllu'
test_path = '/content/drive/MyDrive/UD_English-EWT/en_ewt-ud-test.conllu'

train_sentences = read_conllu(train_path)
dev_sentences = read_conllu(dev_path)
test_sentences = read_conllu(test_path)

print(f'Train samples: {len(train_sentences)}')
print(f'Dev samples: {len(dev_sentences)}')
print(f'Test sentences: {len(test_sentences)}')

Train samples: 12544
Dev samples: 2001
Test sentences: 2077


Build vocab:

In [4]:
pad_token_id = 0
unk_token_id = 1

# Build vocabularies from train_sentences (each token is (form, upos))
words = set()
tags = set()
for sent in train_sentences:
    for form, upos in sent:
        words.add(form)
        tags.add(upos)

# word2idx with special <UNK> and <PAD>
word2idx = {'<UNK>': unk_token_id, '<PAD>': pad_token_id}
word2idx.update({word: idx for idx, word in zip(range(2, len(words)+2), words)})

# tag_to_ix mapping
tag2idx = {tag: i for i, tag in enumerate(sorted(tags))}

print(f"word2idx size (including <UNK>): {len(word2idx)}")
print(f"tag2idx size: {len(tag2idx)}")

word2idx size (including <UNK>): 20202
tag2idx size: 18


### Task 2

Định nghĩa dataset:

In [5]:
from torch.utils.data import Dataset
import torch

class POSDataset(Dataset):
    def __init__(self, sentences, word2idx, tag2idx):
        super().__init__()
        self.sentences = sentences
        self.word2idx = word2idx
        self.tag2idx = tag2idx

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tokens, tags = zip(*sentence)
        # lấy ra tokens ids cho các câu, xử lí out of vocabulary
        sentence_indices = torch.tensor([self.word2idx[token] if token in word2idx.keys() else word2idx['<UNK>'] for token in tokens ], dtype=torch.long)
        tag_indices = torch.tensor([self.tag2idx[tag] for tag in tags], dtype=torch.long)
        return sentence_indices, tag_indices

Định nghĩa collate_fn có nhiệm vụ xử lí padding và khởi tạo data loader:

In [6]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn_with_padding(batch):
    input_ids, tag_ids = zip(*batch)
    padded_sequences = pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
    padded_tags = pad_sequence(tag_ids, batch_first=True, padding_value=-100) # -100 để bỏ qua khi tính loss

    return padded_sequences, padded_tags

train_ds = POSDataset(train_sentences, word2idx, tag2idx)
dev_ds = POSDataset(dev_sentences, word2idx, tag2idx)
test_ds = POSDataset(test_sentences, word2idx, tag2idx)

batch_size = 32
num_workers = 2
train_loader = DataLoader(
    train_ds, batch_size=batch_size, collate_fn=collate_fn_with_padding,
    shuffle=True, generator=torch.Generator().manual_seed(42),
    num_workers=num_workers
)
dev_loader = DataLoader(
    dev_ds, batch_size=batch_size,
    collate_fn=collate_fn_with_padding, shuffle=False, num_workers=num_workers
)
test_loader = DataLoader(
    test_ds, batch_size=batch_size, collate_fn=collate_fn_with_padding,
    shuffle=False, num_workers=num_workers
)

### Task 3

Định nghĩa mô hình:

In [7]:
import torch.nn as nn

class SimpleRNNForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, dropout_p=0.3):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_size ,batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input_seqs):
        emb = self.dropout(self.embedding(input_seqs))
        output, _ = self.rnn(emb) # (batch, seq len, hidden)
        logits = self.fc(output) # fc tự động áp dụng lên dim cuối cùng
        return logits # (batch, seq len, num_classes)


### Task 4 + 5

Khởi tạo mô hình, optimizer và loss function:

In [8]:
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = len(word2idx)
embedding_dim = 128
hidden_size = 256
num_classes = len(tag2idx)
model = SimpleRNNForTokenClassification(vocab_size, embedding_dim, hidden_size, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

Huấn luyện mô hình:

In [9]:
import torch
from tqdm import tqdm

def train_one_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    pbar = tqdm(data_loader, desc='Training', total=len(data_loader))

    for input_ids, tag_ids in pbar:
        input_ids = input_ids.to(device)
        tag_ids = tag_ids.to(device)

        optimizer.zero_grad()

        logits = model(input_ids).view(-1, model.num_classes)
        targets = tag_ids.view(-1)
        loss = criterion(logits, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    epoch_loss = total_loss / len(data_loader)
    return epoch_loss

@torch.no_grad()
def evaluate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    pbar = tqdm(data_loader, desc='Evaluating', total=len(data_loader))
    total_correct = 0
    total_token = 0
    for input_ids, tag_ids in pbar:
        input_ids = input_ids.to(device)
        tag_ids = tag_ids.to(device)
        logits = model(input_ids).view(-1, model.num_classes)
        targets = tag_ids.view(-1)
        loss = criterion(logits, targets)
        total_loss += loss.item()
        # Tính accuracy
        preds = torch.argmax(logits, dim=-1)
        mask = targets != -100
        total_correct += (preds[mask] == targets[mask]).sum().item()
        total_token += mask.sum().item()

    validation_loss = total_loss / len(data_loader)
    accuracy = total_correct / total_token

    return validation_loss, accuracy

In [11]:
epochs = 50
best_accuracy = 0.0
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}:")
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, accuracy = evaluate(model, dev_loader, criterion, device)
    print(f"Train loss: {train_loss:.5f} | Val loss: {val_loss:.5f} | Accuracy: {accuracy:.5f}")

    # lưu lại mô hình tốt nhất dựa trên validation accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Best model saved with accuracy: {best_accuracy:.5f}")

    print("==="*20)


Epoch 1/50:


Evaluating: 100%|██████████| 63/63 [00:00<00:00, 239.26it/s]


Train loss: 0.59643 | Val loss: 0.55782 | Accuracy: 0.82615
Best model saved with accuracy: 0.82615

Epoch 2/50:


Evaluating: 100%|██████████| 63/63 [00:00<00:00, 242.35it/s]


Train loss: 0.50327 | Val loss: 0.52155 | Accuracy: 0.84211
Best model saved with accuracy: 0.84211

Epoch 3/50:


...

Train loss: 0.06637 | Val loss: 0.60910 | Accuracy: 0.89554

Epoch 49/50:


Evaluating: 100%|██████████| 63/63 [00:00<00:00, 245.45it/s]


Train loss: 0.06643 | Val loss: 0.60678 | Accuracy: 0.89589

Epoch 50/50:


Evaluating: 100%|██████████| 63/63 [00:00<00:00, 272.03it/s]

Train loss: 0.06630 | Val loss: 0.60472 | Accuracy: 0.89514


Đánh giá trên tập test:

In [12]:
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print('\n')
print(test_accuracy)

Evaluating: 100%|██████████| 65/65 [00:00<00:00, 155.91it/s]



0.8934381139489195


Hàm dự đoán:

In [13]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
from nltk.tokenize import word_tokenize

def predict(model, tokenizer, sentence, word2idx, idx2word, idx2tag, device):
    model.eval()
    tokens = tokenizer(sentence)
    token_ids = [word2idx[token] if token in word2idx.keys() else word2idx['<UNK>'] for token in tokens]
    input_ids = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(device)
    logits = model(input_ids)
    preds = torch.argmax(logits, dim=-1).squeeze(0)
    predicted_tags = [idx2tag[tag_id] for tag_id in preds.tolist()]
    return list(zip(tokens, predicted_tags))

sample_sentences = [
    "I love NLP",
    "This is Sparta!!!!!",
    "This movie is really interesting",
    "I rate this movie ten out of ten"
]

idx2word = {idx: word for word, idx in word2idx.items()}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

for sent in sample_sentences:
    output = predict(model, word_tokenize, sent, word2idx, idx2word, idx2tag, device)
    print(output)

[('I', 'PRON'), ('love', 'VERB'), ('NLP', 'PROPN')]
[('This', 'DET'), ('is', 'AUX'), ('Sparta', 'NOUN'), ('!', 'PUNCT'), ('!', 'PUNCT'), ('!', 'PUNCT'), ('!', 'PUNCT'), ('!', 'PUNCT')]
[('This', 'DET'), ('movie', 'NOUN'), ('is', 'AUX'), ('really', 'ADV'), ('interesting', 'ADJ')]
[('I', 'PRON'), ('rate', 'VERB'), ('this', 'DET'), ('movie', 'NOUN'), ('ten', 'NUM'), ('out', 'ADP'), ('of', 'ADP'), ('ten', 'NUM')]


### Kết quả (50 epochs):
- Dev accuracy: 89.87%
- Test accuracy: 89.34%